# Hands-on #6. Integer Programming
Recall the team making scenario in the example that we went through during the class. We want to formulate assignment team formation as an integer programming problem and solve it. We have 20 students in the class. Half of the students are in a master’s program and half are in a PhD program. We want to form as many pairs as possible. During the lecture, we designed constraints and the objective function to encode:
- Some students already formed pairs and they have partners
- Some students prefer working alone
- We want to encourage master’s students and PhD students to work together.

Now, students are working on assignment 2. We want to:
- Task #1 (design an objective function). Encourage students to work with a partner who is different from the previous assignment partner.
- Task #2 (design constraints). Prohibit students from working with the same partner.

In [43]:
import pulp
from itertools import product

In [44]:
students = [
    { "id": 0, "program": "master" },
    { "id": 1, "program": "master" },
    { "id": 2, "program": "master" },
    { "id": 3, "program": "master" },
    { "id": 4, "program": "master" },
    { "id": 5, "program": "master" },
    { "id": 6, "program": "master" },
    { "id": 7, "program": "master" },
    { "id": 8, "program": "master" },
    { "id": 9, "program": "master" },
    { "id": 10, "program": "phd" },
    { "id": 11, "program": "phd" },
    { "id": 12, "program": "phd" },
    { "id": 13, "program": "phd" },
    { "id": 14, "program": "phd" },
    { "id": 15, "program": "phd" },
    { "id": 16, "program": "phd" },
    { "id": 17, "program": "phd" },
    { "id": 18, "program": "phd" },
    { "id": 19, "program": "phd" },
]
sids = [s["id"] for s in students]
ss = list(product(sids, sids))

previous_pairs = [
    (0, 10),
    (1, 13),
    (2, 3),
    (4, 5),
    (6, 7),
    (8, 19),
    (9, 12),
    (11, 18),
    (14, 17),
    (15, 16)
]

# Task #1. Design an Objective Function
Encourage students to work with a partner that is different from the previous partner.
Edit the weight function to set the soft constraint.

In [60]:
prob = pulp.LpProblem('TeamMaking-1', pulp.LpMaximize)
pair = pulp.LpVariable.dicts('pair', ss, cat='Binary')

In [61]:
# Constraints
# Each student can only find one partner
for sid in sids:
    prob += pulp.lpSum([pair[sid, sid2] for sid2 in sids]) == 1

# The partnership needs to be symmetric
for _ss in ss:
    prob += pair[_ss[0], _ss[1]] - pair[_ss[1], _ss[0]] == 0

In [62]:
# Objective
# Preference
def weight(sid1, sid2):
    reward = 0

    # It is nice if phd student and non-phd student work together
    program1 = students[sid1]['program']
    program2 = students[sid2]['program']
    if program1 != program2:
        reward += 1

    # It is nice if students work together with another student
    if sid1 != sid2:
        reward += 1

    # It is nice if students work a new partner
    # TODO


    return reward

prob += pulp.lpSum([pair[sid1, sid2] * weight(sid1, sid2) for sid1, sid2 in ss])

In [63]:
# Solve
status = prob.solve()
pulp.LpStatus[status]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kotarohara/repo/Python/web-optimization/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/4_/vrr8kzqn5b9dxsprxn13022m0000gn/T/c1d136376a51492d8e4528f0185873e4-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/4_/vrr8kzqn5b9dxsprxn13022m0000gn/T/c1d136376a51492d8e4528f0185873e4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 425 COLUMNS
At line 2786 RHS
At line 3207 BOUNDS
At line 3608 ENDATA
Problem MODEL has 420 rows, 400 columns and 1160 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 40 - 0.00 seconds
Cgl0004I processed model has 20 rows, 210 columns (210 integer (210 of which binary)) and 400 elements
Cutoff increment increased from 1e-05 to 1.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solut

'Optimal'

In [64]:
for key in ss:
    if pair[key].value() > 0:
        print(pair[key])
        print(students[key[0]])
        print(students[key[1]])
        print()

pair_(0,_10)
{'id': 0, 'program': 'master'}
{'id': 10, 'program': 'phd'}

pair_(1,_11)
{'id': 1, 'program': 'master'}
{'id': 11, 'program': 'phd'}

pair_(2,_15)
{'id': 2, 'program': 'master'}
{'id': 15, 'program': 'phd'}

pair_(3,_13)
{'id': 3, 'program': 'master'}
{'id': 13, 'program': 'phd'}

pair_(4,_18)
{'id': 4, 'program': 'master'}
{'id': 18, 'program': 'phd'}

pair_(5,_19)
{'id': 5, 'program': 'master'}
{'id': 19, 'program': 'phd'}

pair_(6,_16)
{'id': 6, 'program': 'master'}
{'id': 16, 'program': 'phd'}

pair_(7,_17)
{'id': 7, 'program': 'master'}
{'id': 17, 'program': 'phd'}

pair_(8,_14)
{'id': 8, 'program': 'master'}
{'id': 14, 'program': 'phd'}

pair_(9,_12)
{'id': 9, 'program': 'master'}
{'id': 12, 'program': 'phd'}

pair_(10,_0)
{'id': 10, 'program': 'phd'}
{'id': 0, 'program': 'master'}

pair_(11,_1)
{'id': 11, 'program': 'phd'}
{'id': 1, 'program': 'master'}

pair_(12,_9)
{'id': 12, 'program': 'phd'}
{'id': 9, 'program': 'master'}

pair_(13,_3)
{'id': 13, 'program': 'ph

# Task #2 Design Constraints
Force students to work with a partner that is different from the previous partner.
Add hard constraints to achieve this.

In [55]:
prob2 = pulp.LpProblem('TeamMaking-2', pulp.LpMaximize)
pair2 = pulp.LpVariable.dicts('pair', ss, cat='Binary')

In [56]:
# Constraints
# Each student can only find one partner
for sid in sids:
    prob2 += pulp.lpSum([pair2[sid, sid2] for sid2 in sids]) == 1

# The partnership needs to be symmetric
for _ss in ss:
    prob2 += pair2[_ss[0], _ss[1]] - pair2[_ss[1], _ss[0]] == 0


# Students are not allowed to work with the previous partner
# TODO

In [57]:
# Objective
# Preference
def weight(sid1, sid2):
    reward = 0

    # It is nice if phd student and non-phd student work together
    program1 = students[sid1]['program']
    program2 = students[sid2]['program']
    if program1 != program2:
        reward += 1

    # It is nice if students work together with another student
    if sid1 != sid2:
        reward += 1

    # It is nice if students work a new partner
    # TODO


    return reward

prob2 += pulp.lpSum([pair2[sid1, sid2] * weight(sid1, sid2) for sid1, sid2 in ss])

In [58]:
status = prob2.solve()
pulp.LpStatus[status]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kotarohara/repo/Python/web-optimization/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/4_/vrr8kzqn5b9dxsprxn13022m0000gn/T/dd2501b0bb834eb3905d7963581af257-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/4_/vrr8kzqn5b9dxsprxn13022m0000gn/T/dd2501b0bb834eb3905d7963581af257-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 425 COLUMNS
At line 2786 RHS
At line 3207 BOUNDS
At line 3608 ENDATA
Problem MODEL has 420 rows, 400 columns and 1160 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 40 - 0.00 seconds
Cgl0004I processed model has 20 rows, 210 columns (210 integer (210 of which binary)) and 400 elements
Cutoff increment increased from 1e-05 to 1.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solut

'Optimal'

In [59]:
for key in ss:
    if pair2[key].value() > 0:
        print(pair2[key])
        print(students[key[0]])
        print(students[key[1]])
        print()

pair_(0,_10)
{'id': 0, 'program': 'master'}
{'id': 10, 'program': 'phd'}

pair_(1,_11)
{'id': 1, 'program': 'master'}
{'id': 11, 'program': 'phd'}

pair_(2,_15)
{'id': 2, 'program': 'master'}
{'id': 15, 'program': 'phd'}

pair_(3,_13)
{'id': 3, 'program': 'master'}
{'id': 13, 'program': 'phd'}

pair_(4,_18)
{'id': 4, 'program': 'master'}
{'id': 18, 'program': 'phd'}

pair_(5,_19)
{'id': 5, 'program': 'master'}
{'id': 19, 'program': 'phd'}

pair_(6,_16)
{'id': 6, 'program': 'master'}
{'id': 16, 'program': 'phd'}

pair_(7,_17)
{'id': 7, 'program': 'master'}
{'id': 17, 'program': 'phd'}

pair_(8,_14)
{'id': 8, 'program': 'master'}
{'id': 14, 'program': 'phd'}

pair_(9,_12)
{'id': 9, 'program': 'master'}
{'id': 12, 'program': 'phd'}

pair_(10,_0)
{'id': 10, 'program': 'phd'}
{'id': 0, 'program': 'master'}

pair_(11,_1)
{'id': 11, 'program': 'phd'}
{'id': 1, 'program': 'master'}

pair_(12,_9)
{'id': 12, 'program': 'phd'}
{'id': 9, 'program': 'master'}

pair_(13,_3)
{'id': 13, 'program': 'ph